<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week7/Calculating_BERT_score_On_jaccsim_EM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Loading libraries and data

In [ ]:
!pip install evaluate
!pip install bert_score
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm import tqdm
import re
from string import punctuation
import torch
import pandas as pd
from time import time
from evaluate import load
import matplotlib.pyplot as plt
import tensorflow as tf
nltk.download("")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Importing data and necessary preprocessing

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [52]:
df = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv")
df = df.drop("Unnamed: 0", axis = 1)

##### Calculating BERT Scores

In [53]:
bertscore = load("bertscore")

In [55]:
df = df.dropna()
df =df.reset_index()
df = df.drop("index", axis = 1)

In [56]:
df

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
0,700295,700313,how many π bonds are present in ferrocene,how many π are present in ferrocene,2073346,2073271
1,700295,731162,how many π bonds are present in ferrocene,how many π bonds are present in silicon dioxide,2073346,1909343
2,698013,697895,what is covalency of chlorine atom in second e...,what is covalency of chlorine atom in second e...,2073324,2073165
3,697895,697895,calculate value of x y here x and y are the to...,calculate value of x y here x and y are the to...,2073215,2073343
4,697895,698013,in phosphorus acid if x is number of non bondi...,in phosphorus acid if x is number of non bondi...,2073202,2073272
...,...,...,...,...,...,...
46625,511203,511064,find the verb and mention if it is used transi...,find the verb and mention whether it is used t...,1584774,1584751
46626,511203,511064,find the verb and mention if it is used transi...,find the verb and mention if it is used transi...,1584774,1584762
46627,511203,511203,find the verb and mention if it is used transi...,find the verb and mention whether it is used t...,1584774,1457810
46628,511203,453,find the verb and mention if it is used transi...,find the verb and mention if it is used transi...,1584774,1286002


In [58]:
df = df[:100]

GPU based execution for 10k samples ==> 2 m 42 s


In [59]:
t =time()
predictions1 = list(df['ques1'])
references1 = list(df['ques2'])
results1 = bertscore.compute(predictions=predictions1, references=references1, lang="en", rescale_with_baseline=True, verbose = True)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1966.51 seconds, 0.05 sentences/sec
Time taken:  19 s,  1 m


In [61]:
df['bert-scores-precision'] = results1['precision']
df['bert-scores-recall'] = results1['recall']
df['bert-scores-f1'] = results1['f1']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df

In [63]:
df.to_csv("drive/MyDrive/Extramarks Project/Dataset versions/Model1-Semantic/JaccPlusBertEM.csv")